In [ ]:
# 🧰 1) Setup
!rm -rf /kaggle/working/MedVLMBenchPhase1
!git clone https://github.com/dimplek0424/MedVLMBenchPhase1.git
%cd /kaggle/working/MedVLMBenchPhase1

# Deps for CheXzero (their repo + clip)
!pip install --quiet "git+https://github.com/rajpurkarlab/cheXzero.git"
!pip install --quiet "ftfy" "regex" "tqdm"  # common CLIP deps
# If CLIP isn't pulled by cheXzero, also:
# !pip install --quiet "git+https://github.com/openai/CLIP.git"

import os, sys
os.environ["DATA_DIR"]   = "/kaggle/input/chest-xrays-indiana-university"
os.environ["OUTPUT_DIR"] = "/kaggle/working/outputs"

print("python:", sys.executable)
print("REPO_DIR:", os.getcwd())
print("DATA_DIR:", os.getenv("DATA_DIR"))
print("OUTPUT_DIR:", os.getenv("OUTPUT_DIR"))
!ls -alh configs


In [ ]:
# ✅ 2) Sanity
import pathlib, torch
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("runner exists:", pathlib.Path("scripts/projection_chexzero.py").exists())


In [ ]:
# 🚀 3) Run CheXzero projection
import os, sys, subprocess
os.makedirs("/kaggle/working/outputs", exist_ok=True)
os.makedirs("results/projection", exist_ok=True)

cfg  = "configs/dataset_iu_v03_full.yaml"
task = "configs/task_projection_v01.yaml"
out  = "results/projection/iu_v03_full_chexzero.csv"

cmd = [sys.executable, "scripts/projection_chexzero.py",
       "--config", cfg, "--task", task, "--out", out]
print("Running:", " ".join(cmd))
subprocess.check_call(cmd)
print("\n✅ Done. See", out)


In [ ]:
# 🔍 4) Smoke check
import os, pandas as pd
csv_path = "results/projection/iu_v03_full_chexzero.csv"
assert os.path.exists(csv_path), "Output CSV not found."
df = pd.read_csv(csv_path)
print(df.head(3))
print("\nRows:", len(df))


In [ ]:
# 📊 5) Eval
PRED_CSV = "results/projection/iu_v03_full_chexzero.csv"
!python scripts/evaluate_views.py \
  --csv {PRED_CSV} \
  --col_image image \
  --col_p1 p_frontal \
  --col_p2 p_lateral \
  --col_pred pred \
  --outdir results/eval_chexzero

!ls -lh results/eval_chexzero
!zip -r /kaggle/working/eval_chexzero_artifacts.zip results/eval_chexzero
